<a href="https://colab.research.google.com/github/Karlosag2020/SIM_YAC_AGUIRRE_UPSE2023/blob/main/COMPARACION%20DE%20ESQUEMAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPARACION DE ESQUEMAS**

In [29]:
# importar librerias
import numpy as np
import math

In [30]:
#DATOS DE YACIMIENTO
L=10000
dx=2500
Tiempo=100000
DT=1
K=50 #Permeabilida MD
por=0.2 #porosidad
vis=1 # viscosidad
Beta=1
CompTotal=1**(-6) #compresibilidad total ps1-1
A=200000 #ft^2


In [31]:
Alfa= K/(vis*por*CompTotal)
Alfa
Eta=(Alfa*DT)/(dx**2)
Eta

4e-05

In [50]:
#Esquema Explicito
def Presdt(P0,P1,P2, Eta): #misma fucnicon para el sistema explicito
  Pdt=P1+Eta*(P0-2*P1+P2)
  return(Pdt)

Pexplicita=np.zeros((Tiempo//DT,L//dx))

In [51]:
#Codiciones iniciales
Pyac=1000 #Presion de yacimiento
Piz=2000 #Presion de la frontera izquierda
Pder=0 #Presion de la frontera derecha
Pin=np.ones(L//dx) #vector de condiciones 
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder

Pin

array([2000., 1000., 1000.,    0.])

In [57]:
t=DT
Pexplicita[0]=Pin 
Pexplicita.shape
while t<Tiempo: #ciclo de tiempo
  for i in range(0,len(Pin)-1): #ciclo de espacio
    if i==0: #condicion inicial
      Pres[i]=Pin[0] #presion de la frontera
    elif i==len(Pin)-1: #condicion de frontera derecha
      Pres[i]=Pin[len(Pin)-1] #Vector de resultado evaluado en ese mismo punto
    else:  
      Pres[i]=Presdt(Pin[i-1],Pin[i],Pin[i+1],Eta) #guardar en la matriz, indice de la fila, 
  Pexplicita[t//DT]=Pres   
  t=t+DT    #cambio de tiempo
  Pin=Pres    #inicio de ciclo

In [58]:
#Esquema Explicito
Pimplicita=np.zeros((Tiempo//DT,L//dx))

Pin=np.ones(L//dx) #vector de condiciones 
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
N=L//dx #Numero de Nodos
Matriz_resolucion=np.zeros([N-2,N-2]) #creacion de matriz cuadrada


In [59]:
for i in range(N-2): #Matriz de resolucion,(N-2)debido a que se debe restar por hallarse en la frontera, valores si hay a la izquierda a la derecha no
  Matriz_resolucion[i,i]=1+2*Eta #los valores de la diagonal son los mismo por lo tanto si i=j entonces i=i, indice de la matris son iguales
  if i>0: # llenar las casillas a la izquierda
    Matriz_resolucion[i,i-1]=-Eta
  if i<(N-3): # si tiene valor a la derecha, el numero de nodos es N, por lo tanto N-3 es debido a que debe buscar un valor 
    Matriz_resolucion[i,i+1]=-Eta #llena los valores a la 


In [60]:
Pcal=np.zeros([N-2]) #presion calculada, vector vacio del tamaño N-2
Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1]) #

In [61]:
Pcal #valores calculados de la diagonal

array([999.9600016, 999.9600016])

In [66]:
#condiciones de forntera
Pres=np.concatenate(([Piz],Pcal,[Pder])) #
Pres

TypeError: ignored

In [ ]:
Pimplicita[0]=Pin
t=DT
while t<Tiempo: #Ciclo en el tiempo
  Pcal=np.linalg.solve(Matriz_resolucion,Pin[i:-1])
  Pres=np.concatenate(([Piz],Pcal,[Pder]))
  Pimplicita[t//DT]=Pres
  Pin=Prest=t+DT